In [1]:
import pandas as pd
DATASET_PATH = "dataset/2024_dataset.json"

In [2]:
df = pd.read_json(DATASET_PATH, lines=True)
df.head()

,text,contains_driver,drivers
0,Fail of the day: Stroll. AKA Latifi & Magnusse...,True,[]
1,This why Ricc career is over... we see again t...,True,[]
2,0:19,False,[]
3,Hall Cynthia Jackson Kimberly Lewis Sharon,True,[]
4,🔥,False,[]


In [3]:
df["driver_count"] = df["drivers"].apply(len)
df.head()
df["driver_count"].value_counts()

driver_count
0     67569
1     19082
2      5860
3       972
4       214
5        61
6        19
7        11
9         2
8         2
10        1
Name: count, dtype: int64

In [4]:
df = df[df["driver_count"] <= 1]
print(df["driver_count"].value_counts())
df.head()

driver_count
0    67569
1    19082
Name: count, dtype: int64


,text,contains_driver,drivers,driver_count
0,Fail of the day: Stroll. AKA Latifi & Magnusse...,True,[],0
1,This why Ricc career is over... we see again t...,True,[],0
2,0:19,False,[],0
3,Hall Cynthia Jackson Kimberly Lewis Sharon,True,[],0
4,🔥,False,[],0


In [5]:
df.drivers.value_counts()

drivers
[]                    67569
[Max Verstappen]       4052
[Lando Norris]         3035
[Lewis Hamilton]       2448
[Carlos Sainz]         1592
[Sergio Perez]         1360
[Charles Leclerc]      1338
[Oscar Piastri]        1262
[George Russell]       1052
[Fernando Alonso]       783
[Yuko Tsunoda]          708
[Lance Stroll]          271
[Kevin Magnussen]       233
[Esteban Ocon]          204
[Daniel Ricciardo]      179
[Valtteri Bottas]       157
[Alex Albon]            147
[Logan Sargeant]        112
[Niko Hulkenberg]        66
[Guan Yu Zhou]           61
[Pierre Gasly]           22
Name: count, dtype: int64

In [43]:
df = df[["text", "drivers"]]
df.head()

,text,drivers
0,Fail of the day: Stroll. AKA Latifi & Magnusse...,[]
1,This why Ricc career is over... we see again t...,[]
2,0:19,[]
3,Hall Cynthia Jackson Kimberly Lewis Sharon,[]
4,🔥,[]


In [26]:
df.to_json("dataset/training_dataset.json", orient="records", lines=False)

In [24]:
df_part1 = df.iloc[:len(df)//2]
df_part2 = df.iloc[len(df)//2:]

# Define the paths for the new JSON files
json_path_part1 = 'training_dataset_part1.json'
json_path_part2 = 'training_dataset_part2.json'

# Save each part to a separate JSON file
df_part1.to_json(json_path_part1, orient="records", lines=True)
df_part2.to_json(json_path_part2, orient="records", lines=True)

In [25]:
print(len(df_part1), len(df_part2))
print(len(df))

43325 43326
86651


In [58]:
from textblob import TextBlob
import nltk 

In [101]:
def get_polarity(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

In [77]:
df['polarity'] = df['text'].apply(get_polarity)

In [102]:
string = """What a  that he  himself for his below average performances.

Anyways, he should know that was his  and last home race.

He definitely won't be driving F1 anymore next year."""
print(get_polarity(string))

-0.0375


In [82]:
fake = df[df.polarity.isin([-0.02, 0.02])].head(20)
for index, row in fake.iterrows():
    print(row.polarity, row['text'])
    print("---")

0.02 What a joke that he celebrates himself for his below average performances.

Anyways, he should know that was his first and last home race.

He definitely won't be driving F1 anymore next year.
---
0.02 Basically the only action you saw of Norris the entire highlight package was when he crossed the line in second place?? Last night, I checked if I needed to set my alarm to wake up for the race by looking at the weather forecast. No rain = lie in.
---
0.02 Double double pit stops with 1.9/2.0 from red bull. WOW. Team moral and work is from another world.
---
0.02 RB is same parts as RedBull, but not same race operation as RedBull😂
why didn't only Yuki sets soft tyre in last stint?
---
0.02 @FLAXXION  It also because its one of the only action parts towards the last race. 4 overtakes in the back of the grid and that brief wall touch.
---


In [107]:
import tensorflow as tf
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis", framework="tf")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [ ]:
def get_polarity_transformers(text):
    result = sentiment_pipeline(text)
    return result[0]['label']

df['polarity_transformers'] = df['text'].apply(get_polarity_transformers)

In [ ]:
fake2 = df[df.polarity_transformers.isin([-0.05, 0.05])].head(20)
for index, row in fake2.iterrows():
    print(row.polarity, row['text'])
    print("---")

In [ ]:
result = sentiment_pipeline("Get in Lando! Fookin get in son!")
print(result)